In [1]:
from pyspark.sql import Window, SparkSession
from pyspark.sql.functions import explode, collect_list, size, col, row_number,lit,udf,sum,split,when,min,max
from pyspark.sql.types import ArrayType,IntegerType
import os


In [2]:
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))
sparkSession = SparkSession.builder.enableHiveSupport().master("local [4]").getOrCreate()


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [3]:
base = sparkSession.read.text("/data/twitter/twitter_sample.txt")
n=100
basedf = base.select(split(col("value"),"\t").getItem(1).alias("e1"),split(col("value"),"\t").getItem(0).alias("e0")).repartition(n).cache()

In [4]:
d = [{'e': '12','dist': 0}]

distanceDF = spark.createDataFrame(d).select("e","dist").repartition(n)
#distanceDF=sparkSession.read.csv("7dist")

/usr/local/spark/python/pyspark/sql/session.py:320: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [ ]:
"""
dis = 0

for i in xrange(10):
    
    contestant = basedf.join(distanceDF,basedf.e1==distanceDF.e).select("e0",lit(dis+1).alias("dist"))
    new_dist = distanceDF.alias("a").join(contestant.alias("b"),distanceDF.e==contestant.e0,"fullouter") \
        .select(when(col("b.e0").isNotNull(),col("b.e0")).otherwise(col("a.e")).alias("e"), \
                when(col("a.dist").isNotNull(),col("a.dist")).otherwise(col("b.dist")).alias("dist")).repartition(100).cache()
    cnt = new_dist.filter(col("dist")==dis+1).count()
    print i,cnt
    if cnt >0:
        dis += 1
        distanceDF = new_dist
    else:
        break
    
"""
contestant = basedf.join(distanceDF,basedf.e1==distanceDF.e).select("e0",lit(1).alias("dist"))
new_dist = distanceDF.alias("a").union(contestant.alias("b")).groupBy("e").agg(min("dist").alias("dist"))
#new_dist = distanceDF.alias("a").union(contestant.alias("b")).groupBy("e").agg(min("dist").alias("dist")).repartition(n)
#new_dist.show()
#.repartition(100)
print distanceDF.rdd.getNumPartitions()
print contestant.rdd.getNumPartitions()
print new_dist.rdd.getNumPartitions()
new_dist.show()


In [6]:
dis = 0

for i in xrange(10):
    contestant = basedf.join(distanceDF,basedf.e1==distanceDF.e).select("e0",(col("dist")+1).alias("dist"))
    new_dist = distanceDF.alias("a").union(contestant.alias("b")).repartition(n).groupBy("e").agg(min("dist").alias("dist")).cache()
    cnt = new_dist.filter(col("dist")==dis+1).count()
    print i,cnt,"DistanceDF size",new_dist.count()
    if cnt >0:
        dis += 1
        distanceDF = new_dist
    else:
        break
    

KeyboardInterrupt: 

In [ ]:
contestant = basedf.join(distanceDF,basedf.e1==distanceDF.e).select("e0","dist+1".alias("dist"))


new_dist = distanceDF.alias("a").join(contestant.alias("b"),distanceDF.e==contestant.e0,"fullouter") \
    .select(when(col("b.e0").isNotNull(),col("b.e0")).otherwise(col("a.e")).alias("e"), \
            when(col("a.dist").isNotNull(),col("a.dist")).otherwise(col("b.dist")).alias("dist")).repartition(100).cache()
cnt = new_dist.filter(col("dist")==dis+1).count()

In [6]:
dis=0
contestant = basedf.alias("a").join(distanceDF.alias("b"),basedf.e1==distanceDF.e).select("e0",lit(dis+1).alias("dist"))
new_dist = distanceDF.union(contestant).groupBy("e").agg(min("dist").alias("dist")).repartition(100)
new_dist.printSchema()

root
 |-- e: string (nullable = true)
 |-- dist: long (nullable = true)



In [ ]:
# contestant = basedf.join(distanceDF,basedf.e1==distanceDF.e).select("e0",(col("dist")+1).alias("dist")).repartition(100)
# new_dist = distanceDF.alias("old").join(contestant.alias("new"),distanceDF.e==contestant.e0,"fullouter") \
#     .select(when(col("old.e").isNotNull(),col("old.e")).otherwise(col("new.e0")).alias("e"), \
#             when(col("old.dist").isNotNull(),col("old.dist")).otherwise(col("new.dist")).alias("dist")).repartition(100).cache()
dis = 0

for i in xrange(9):
    print "processing", i
    contestant = basedf.alias("a").join(distanceDF.alias("b"),basedf.e1==distanceDF.e).select("e0",lit(dis+1).alias("dist"))
    new_dist = distanceDF.union(contestant).groupBy("e").agg(min("dist").alias("dist")).repartition(100)
    cnt = new_dist.filter(col("dist")==dis+1).count()
    print "iteration",i,cnt
    if cnt >0:
        dis += 1
        distanceDF = new_dist
    else:
        new_dist.filter("e == 34").show()
    
    

processing 0
0 4 DistanceDF size
processing 1
1 3 DistanceDF size
processing 2
2 9 DistanceDF size
processing 3
3 15 DistanceDF size
processing 4
4 15 DistanceDF size
processing 5
5 14 DistanceDF size
processing 6
6 6 DistanceDF size
processing 7
7 4 DistanceDF size
processing 8
8 0 DistanceDF size
+---+----+
|  e|dist|
+---+----+
| 34|   8|
+---+----+

